In [17]:
import pandas as pd
import ast

estoque = pd.read_csv('spreadsheets/soma_estoque.csv')
pedidos = pd.read_csv('spreadsheets/pedidos.csv')
precos = pd.read_csv('spreadsheets/infos_de_precos.csv')
fornecedores = pd.read_csv('spreadsheets/fornecedores.csv')
produtos = pd.read_csv('spreadsheets/produtos.csv')

In [2]:
pedidos['produtos']

0    [{'id': 2, 'quantidade': 4}, {'id': 3, 'quanti...
1    [{'id': 1, 'quantidade': 2}, {'id': 2, 'quanti...
Name: produtos, dtype: object

In [3]:
pedidos['produtos'] = pedidos['produtos'].apply(ast.literal_eval) #Transformando em lista
print(pedidos['produtos'].apply(type).unique()) #Verificando tipo de dados da coluna

[<class 'list'>]


In [4]:
# Explodindo a lista para separar cada produto em uma linha
a = pedidos.explode('produtos').reset_index(drop=True)

# Transformando os dicionários em colunas separadas
a = a.join(pd.DataFrame(a.pop('produtos').tolist()).add_prefix('prod_'))
a

,cliente,id,status,prod_id,prod_quantidade
0,Empresa A,1,Pago,2,4
1,Empresa A,1,Pago,3,3
2,Empresa B,2,Pendente,1,2
3,Empresa B,2,Pendente,2,5


In [5]:
estoque

,idP,estoque
0,1,25
1,2,50
2,3,30
3,4,20


In [6]:
merged = pd.merge(a, estoque, left_on='prod_id', right_on='idP', how='inner')
merged

,cliente,id,status,prod_id,prod_quantidade,idP,estoque
0,Empresa A,1,Pago,2,4,2,50
1,Empresa A,1,Pago,3,3,3,30
2,Empresa B,2,Pendente,1,2,1,25
3,Empresa B,2,Pendente,2,5,2,50


In [12]:
merged['estoque'] = merged['estoque'] - merged['prod_quantidade']
merged

,cliente,id,status,prod_id,prod_quantidade,idP,estoque
0,Empresa A,1,Pago,2,4,2,46
1,Empresa A,1,Pago,3,3,3,27
2,Empresa B,2,Pendente,1,2,1,23
3,Empresa B,2,Pendente,2,5,2,45


In [15]:
estoque.update(merged[['idP','estoque']]) # Garante que as atualizações sejam feitas sem duplicações

In [16]:
estoque

,idP,estoque
0,2,46
1,3,27
2,1,23
3,2,45


In [19]:
produtos

,idP,nome,marca,estoque_minimo,preco_venda
0,1,Notebook,Dell,5,4000.0
1,2,Mouse,Logitech,10,25.0
2,3,Teclado,Microsoft,8,80.0
3,4,Monitor,Samsung,3,600.0
4,5,Notebook DELL,Dell,5,4000.0
5,6,mouse sem fio,logitech,10,25.0
6,7,Teclado GAMER,microsoft,8,80.0
7,8,monitor full hd,Samsung,3,600.0
8,9,NOTEBOOK dell,DELL,5,4100.0


In [24]:
# Title case e remoção de espaços extras
columns = ['Nome','Marca']
produtos[columns] = produtos[columns].apply(lambda x: x.str.title())
produtos

,Idp,Nome,Marca,Estoque_Minimo,Preco_Venda
0,1,Notebook,Dell,5,4000.0
1,2,Mouse,Logitech,10,25.0
2,3,Teclado,Microsoft,8,80.0
3,4,Monitor,Samsung,3,600.0
4,5,Notebook Dell,Dell,5,4000.0
5,6,Mouse Sem Fio,Logitech,10,25.0
6,7,Teclado Gamer,Microsoft,8,80.0
7,8,Monitor Full Hd,Samsung,3,600.0
8,9,Notebook Dell,Dell,5,4100.0


In [31]:
# Removendo colunas com dados redundantes
produtos.sort_values(by=['Marca'])

,Idp,Nome,Marca,Estoque_Minimo,Preco_Venda
0,1,Notebook,Dell,5,4000.0
4,5,Notebook Dell,Dell,5,4000.0
8,9,Notebook Dell,Dell,5,4100.0
1,2,Mouse,Logitech,10,25.0
5,6,Mouse Sem Fio,Logitech,10,25.0
2,3,Teclado,Microsoft,8,80.0
6,7,Teclado Gamer,Microsoft,8,80.0
3,4,Monitor,Samsung,3,600.0
7,8,Monitor Full Hd,Samsung,3,600.0


In [33]:
prod = produtos.drop_duplicates(subset=['Nome','Marca'], keep='first')
prod

,Idp,Nome,Marca,Estoque_Minimo,Preco_Venda
0,1,Notebook,Dell,5,4000.0
1,2,Mouse,Logitech,10,25.0
2,3,Teclado,Microsoft,8,80.0
3,4,Monitor,Samsung,3,600.0
4,5,Notebook Dell,Dell,5,4000.0
5,6,Mouse Sem Fio,Logitech,10,25.0
6,7,Teclado Gamer,Microsoft,8,80.0
7,8,Monitor Full Hd,Samsung,3,600.0
